<a href="https://colab.research.google.com/github/MrWou/Script_Frazioni_OSM/blob/main/Estrazione_Frazioni_ER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌍 GeoFrazioni ER: Scraper Resiliente per OpenStreetMap

**Obiettivo:** Creare un dataset preciso e georeferenziato di tutte le frazioni dell'Emilia-Romagna, risolvendo problemi di omonimia (es. Lugo in Spagna vs Lugo in Italia) e timeout dei server.

**Metodo:**
1.  **Geocoding (Nominatim):** Identifica il perimetro esatto (Bounding Box) di ogni comune.
2.  **Estrazione (Overpass API):** Scarica le frazioni solo all'interno di quel perimetro.
3.  **Resilienza:** Implementa un sistema multi-server con rotazione automatica in caso di errori 504/Connection Error.

---

## 📍 Step 1: Geocoding dei Comuni (Nominatim)
In questa fase interroghiamo l'API di **Nominatim** per ottenere le coordinate di confine (Bounding Box) di ogni comune presente nel file CSV ISTAT.

**Perché questo passaggio?**
Serve a creare una "gabbia geografica" sicura. Invece di chiedere a Overpass "Dammi le frazioni di Lugo", gli chiederemo "Dammi le frazioni che stanno tra la Latitudine X e Y". Questo impedisce errori di omonimia (es. evitare di scaricare dati da Lugo in Spagna o Ravenna negli USA).

In [3]:
import requests
import pandas as pd
import time

# --- CONFIGURAZIONE ---
INPUT_FILE = "Codice ISTAT Comuni e Provincie ER_update_07_2024.csv"
OUTPUT_GEO_FILE = "Comuni_ER_Georeferenziati.csv"

HEADERS = {
    'User-Agent': 'Script_Frazioni_ER/2.0 (contatto@esempio.com)'
}

def get_geo_info(city, province):
    # Forziamo la ricerca in Emilia-Romagna, Italia per evitare omonimie mondiali
    query = f"{city}, {province}, Emilia-Romagna, Italia"

    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': query,
        'format': 'json',
        'limit': 1,
        'polygon_geojson': 0
    }

    try:
        r = requests.get(url, params=params, headers=HEADERS)
        r.raise_for_status()
        data = r.json()

        if data:
            res = data[0]
            # Nominatim restituisce bbox come [min_lat, max_lat, min_lon, max_lon]
            bbox = res['boundingbox']
            return {
                'osm_id': res.get('osm_id'),
                'lat_centro': res.get('lat'),
                'lon_centro': res.get('lon'),
                'min_lat': bbox[0],
                'max_lat': bbox[1],
                'min_lon': bbox[2],
                'max_lon': bbox[3]
            }
    except Exception as e:
        print(f"Errore per {city}: {e}")

    return None

# --- ESECUZIONE ---
try:
    df = pd.read_csv(INPUT_FILE)
    geo_data = []

    print(f"Inizio georeferenziazione di {len(df)} comuni...")

    for index, row in df.iterrows():
        comune = row["Denominazione Comune"]
        provincia = row["Provincia"]

        print(f"[{index+1}] Geocoding: {comune}...", end=" ", flush=True)

        info = get_geo_info(comune, provincia)

        if info:
            print("OK")
            geo_data.append({
                'Provincia': provincia,
                'Comune': comune,
                **info
            })
        else:
            print("FALLITO")

        # IMPORTANTE: Nominatim richiede 1 secondo di pausa tra le richieste
        time.sleep(1.1)

    # Salvataggio file intermedio
    if geo_data:
        df_geo = pd.DataFrame(geo_data)
        df_geo.to_csv(OUTPUT_GEO_FILE, index=False)
        print(f"\nFase 1 completata. File salvato: {OUTPUT_GEO_FILE}")
    else:
        print("Nessun dato georeferenziato.")

except FileNotFoundError:
    print(f"File {INPUT_FILE} non trovato.")

Inizio georeferenziazione di 330 comuni...
[1] Geocoding: Agazzano... OK
[2] Geocoding: Alseno... OK
[3] Geocoding: Besenzone... OK
[4] Geocoding: Bettola... OK
[5] Geocoding: Bobbio... OK
[6] Geocoding: Borgonovo Val Tidone... OK
[7] Geocoding: Cadeo... OK
[8] Geocoding: Calendasco... OK
[9] Geocoding: Caorso... OK
[10] Geocoding: Carpaneto Piacentino... OK
[11] Geocoding: Castell'Arquato... OK
[12] Geocoding: Castel San Giovanni... OK
[13] Geocoding: Castelvetro Piacentino... OK
[14] Geocoding: Cerignale... OK
[15] Geocoding: Coli... OK
[16] Geocoding: Corte Brugnatella... OK
[17] Geocoding: Cortemaggiore... OK
[18] Geocoding: Farini... OK
[19] Geocoding: Ferriere... OK
[20] Geocoding: Fiorenzuola d'Arda... OK
[21] Geocoding: Gazzola... OK
[22] Geocoding: Gossolengo... OK
[23] Geocoding: Gragnano Trebbiense... OK
[24] Geocoding: Gropparello... OK
[25] Geocoding: Lugagnano Val d'Arda... OK
[26] Geocoding: Monticelli d'Ongina... OK
[27] Geocoding: Morfasso... OK
[28] Geocoding: Ottone.

## 📡 Step 2: Estrazione Frazioni (Overpass Multi-Server)
Utilizziamo i Bounding Box calcolati nello step precedente per scaricare i nodi `place=village` (frazioni) e `place=suburb` (per le zone costiere, escludendo i capoluoghi).

**Caratteristiche di resilienza:**
* **Rotazione Server:** Lo script possiede una lista di server (Tedesco, Francese, Russo, Kumi). Se il principale va in timeout, passa automaticamente al successivo.
* **Anti-Ban:** Gestione automatica dell'errore `429 Too Many Requests` con pause incrementali.
* **No-Regex:** Query ottimizzata per ridurre il carico sulla CPU del server OSM.

In [9]:
import requests
import pandas as pd
import time
import random

# --- CONFIGURAZIONE ---
INPUT_GEO_FILE = "Comuni_ER_Georeferenziati.csv"
OUTPUT_FINAL_FILE = "Frazioni_ER_Finali_Complete.csv"

CITIES_NO_SUBURBS = [
    "Bologna", "Modena", "Parma", "Reggio nell'Emilia", "Reggio Emilia",
    "Ferrara", "Piacenza", "Ravenna", "Rimini", "Forlì", "Cesena"
]

# LISTA SERVER DI ROTAZIONE
# Se uno fallisce, lo script prova il successivo.
OVERPASS_SERVERS = [
    "https://overpass-api.de/api/interpreter",       # Principale (Germania)
    "https://overpass.private.coffee/api/interpreter", # Kumi Systems (Molto affidabile)
    "https://maps.mail.ru/osm/tools/overpass/api/interpreter" # Russia (Spesso libero)
]

def get_fractions_in_bbox(bbox, city_name, use_suburbs=True, attempt=1, server_index=0):

    # Se abbiamo finito i server, ci arrendiamo
    if server_index >= len(OVERPASS_SERVERS):
        print(f" [!] Tutti i server hanno fallito per {city_name}.")
        return []

    current_server = OVERPASS_SERVERS[server_index]

    # Costruiamo la query
    parts = [f'node["place"="village"]({bbox["min_lat"]},{bbox["min_lon"]},{bbox["max_lat"]},{bbox["max_lon"]});']
    if use_suburbs:
        parts.append(f'node["place"="suburb"]({bbox["min_lat"]},{bbox["min_lon"]},{bbox["max_lat"]},{bbox["max_lon"]});')

    query = f"[out:json][timeout:90];\n(\n" + "\n".join(parts) + "\n);\nout body;"

    try:
        # Timeout connessione
        r = requests.get(current_server, params={'data': query}, timeout=100)

        # 1. RATE LIMIT (429) -> Aspetta e riprova sullo STESSO server
        if r.status_code == 429:
            wait_time = 10 * attempt
            print(f" (Rate Limit su {server_index+1} - attendo {wait_time}s)...", end=" ")
            time.sleep(wait_time)
            return get_fractions_in_bbox(bbox, city_name, use_suburbs, attempt + 1, server_index)

        # 2. SERVER ERROR (5xx) -> Prova il PROSSIMO server
        if r.status_code >= 500:
            print(f" [Err {r.status_code} su Server {server_index+1}] Cambio server...", end=" ")
            return get_fractions_in_bbox(bbox, city_name, use_suburbs, 1, server_index + 1)

        r.raise_for_status()
        data = r.json()

        fractions = []
        for el in data.get('elements', []):
            if 'tags' in el and 'name' in el['tags']:
                fractions.append({
                    'Comune': city_name, # Aggiungiamo qui per comodità
                    'Frazione': el['tags']['name'],
                    'Tipologia': el['tags'].get('place'),
                    'Latitudine': el['lat'],
                    'Longitudine': el['lon']
                })
        return fractions

    except (requests.exceptions.RequestException, Exception) as e:
        # 3. ERRORE CONNESSIONE (DNS, Timeout) -> Prova il PROSSIMO server
        print(f" [ErrConn su Server {server_index+1}] Cambio server...", end=" ")
        time.sleep(1)
        return get_fractions_in_bbox(bbox, city_name, use_suburbs, 1, server_index + 1)

# --- ESECUZIONE ---
try:
    df_geo = pd.read_csv(INPUT_GEO_FILE)
    all_results = []

    total = len(df_geo)
    print(f"Inizio estrazione Multi-Server per {total} comuni...")
    print("-" * 60)

    for index, row in df_geo.iterrows():
        comune = row["Comune"]
        provincia = row["Provincia"]
        use_suburbs = comune not in CITIES_NO_SUBURBS

        print(f"[{index+1}/{total}] {comune}...", end=" ", flush=True)

        frazioni = get_fractions_in_bbox(row, comune, use_suburbs=use_suburbs)

        if frazioni:
            print(f"-> {len(frazioni)}")
            # Arricchiamo i dati con la provincia
            for f in frazioni:
                f['Provincia'] = provincia
            all_results.extend(frazioni)
        else:
            print("-> 0")

        # Pausa standard tra comuni
        time.sleep(1.5)

    if all_results:
        df_final = pd.DataFrame(all_results)
        cols = ['Provincia', 'Comune', 'Frazione', 'Tipologia', 'Latitudine', 'Longitudine']
        df_final = df_final.reindex(columns=cols)
        df_final.to_csv(OUTPUT_FINAL_FILE, index=False)
        print("-" * 60)
        print(f"FINITO! Dati salvati in: {OUTPUT_FINAL_FILE}")
    else:
        print("Nessuna frazione trovata.")

except FileNotFoundError:
    print(f"Errore: File {INPUT_GEO_FILE} mancante. Esegui prima lo Script 1!")

Inizio estrazione Multi-Server per 330 comuni...
------------------------------------------------------------
[1/330] Agazzano... -> 1
[2/330] Alseno... -> 5
[3/330] Besenzone... -> 3
[4/330] Bettola... -> 7
[5/330] Bobbio...  (Rate Limit su 1 - attendo 10s)...  (Rate Limit su 1 - attendo 20s)... -> 5
[6/330] Borgonovo Val Tidone... -> 3
[7/330] Cadeo... -> 2
[8/330] Calendasco... -> 9
[9/330] Caorso... -> 5
[10/330] Carpaneto Piacentino... -> 3
[11/330] Castell'Arquato... -> 4
[12/330] Castel San Giovanni... -> 2
[13/330] Castelvetro Piacentino... -> 10
[14/330] Cerignale... -> 2
[15/330] Coli... -> 2
[16/330] Corte Brugnatella... -> 3
[17/330] Cortemaggiore... -> 5
[18/330] Farini... -> 3
[19/330] Ferriere... -> 2
[20/330] Fiorenzuola d'Arda... -> 3
[21/330] Gazzola... -> 6
[22/330] Gossolengo... -> 2
[23/330] Gragnano Trebbiense... -> 7
[24/330] Gropparello... -> 1
[25/330] Lugagnano Val d'Arda... -> 4
[26/330] Monticelli d'Ongina... -> 8
[27/330] Morfasso... -> 10
[28/330] Ottone..

## ✅ Conclusione
Il processo è terminato. Il file finale pulito e georeferenziato è stato salvato come:
`Frazioni_ER_Finali_Complete.csv`

Puoi scaricarlo dalla barra laterale di sinistra (icona Cartella).